In [ ]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-s1pdo0wy
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-s1pdo0wy
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━

In [ ]:
!sudo apt update && sudo apt install ffmpeg

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,321 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,664 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-s

In [ ]:
!pip install watchdog

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.7 MB/s eta 0:00:00


In [ ]:
import os
import whisper
import json
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
SUPPORTED_FORMATS = {'.mp3', '.wav', '.mp4', '.mkv', '.mov', '.flv', '.aac', '.m4a'}
LOG_FILE = "processed_files.json"
def load_processed_files():
    if os.path.exists(LOG_FILE):
        with open(LOG_FILE, "r") as f:
            return set(json.load(f))
    return set()
def save_processed_files(processed_files):
    with open(LOG_FILE, "w") as f:
        json.dump(list(processed_files), f)

In [ ]:
def transcribe_audio(file_path, processed_files):
    """ Transcribes an audio/video file and saves the transcript """
    if file_path in processed_files:
        print(f"Skipping already processed file: {file_path}")
        return

    print(f"Processing: {file_path}")
    model = whisper.load_model("base")  # Use 'small', 'medium', or 'large' for better accuracy
    result = model.transcribe(file_path)

    # Save transcript
    transcript_file = file_path.rsplit('.', 1)[0] + ".txt"
    with open(transcript_file, "w", encoding="utf-8") as f:
        f.write(result["text"])

    print(f"Transcription saved: {transcript_file}")

    # Mark file as processed
    processed_files.add(file_path)
    save_processed_files(processed_files)


In [ ]:
def transcribe_audio(file_path, processed_files):
    """ Transcribes an audio/video file and saves the transcript """
    if file_path in processed_files:
        print(f"Skipping already processed file: {file_path}")
        return

    print(f"Processing: {file_path}")
    model = whisper.load_model("base")  # Use 'small', 'medium', or 'large' for better accuracy
    result = model.transcribe(file_path)

    # Save transcript
    transcript_file = file_path.rsplit('.', 1)[0] + ".txt"
    with open(transcript_file, "w", encoding="utf-8") as f:
        f.write(result["text"])

    print(f"Transcription saved: {transcript_file}")

    # Mark file as processed
    processed_files.add(file_path)
    save_processed_files(processed_files)


In [ ]:
class MediaFileHandler(FileSystemEventHandler):
    def __init__(self, processed_files):
        self.processed_files = processed_files

    def on_created(self, event):
        if not event.is_directory and any(event.src_path.lower().endswith(ext) for ext in SUPPORTED_FORMATS):
            transcribe_audio(event.src_path, self.processed_files)

def scan_directory(root_dir, processed_files):
    """ Recursively scans a directory for unprocessed media files """
    for dirpath, _, filenames in os.walk(root_dir):
        for file in filenames:
            if any(file.lower().endswith(ext) for ext in SUPPORTED_FORMATS):
                file_path = os.path.join(dirpath, file)
                transcribe_audio(file_path, processed_files)

def watch_directory(directory, processed_files):
    """ Monitors a directory for new media files in real-time """
    event_handler = MediaFileHandler(processed_files)
    observer = Observer()
    observer.schedule(event_handler, directory, recursive=True)
    observer.start()
    print(f"Watching directory: {directory}")

    try:
        while True:
            pass  # Keep the script running
    except KeyboardInterrupt:
        observer.stop()
    observer.join()


In [ ]:
if __name__ == "__main__":
    target_directory = "/content/"  # Ensure it's a folder

    # Initialize processed_files to track already transcribed files
    processed_files = set()

    # Scan existing files and transcribe them
    scan_directory(target_directory, processed_files)

    # Start real-time monitoring
    watch_directory(target_directory, processed_files)


Processing: /content/Kabira Yeh Jawaani Hai Deewani 128 Kbps.mp3


100%|███████████████████████████████████████| 139M/139M [00:03<00:00, 38.4MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Transcription saved: /content/Kabira Yeh Jawaani Hai Deewani 128 Kbps.txt
Processing: /content/Power_English_Update (1).mp3


/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


Transcription saved: /content/Power_English_Update (1).txt
Watching directory: /content/


In [ ]:
import time

def watch_directory_for_limited_time(target_directory, processed_files, duration=60):
    """Watch directory for new files but stop after 'duration' seconds."""
    start_time = time.time()
    while time.time() - start_time < duration:
        watch_directory(target_directory, processed_files)
    print("🛑 Stopping real-time monitoring after", duration, "seconds.")

if __name__ == "__main__":
    target_directory = "/content/"
    processed_files = set()

    # Process existing files
    scan_directory(target_directory, processed_files)

    # Monitor for new files but stop after 60 seconds
    watch_directory_for_limited_time(target_directory, processed_files, duration=60)


Processing: /content/Kabira Yeh Jawaani Hai Deewani 128 Kbps.mp3


/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


Transcription saved: /content/Kabira Yeh Jawaani Hai Deewani 128 Kbps.txt
Processing: /content/Power_English_Update (1).mp3


/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


Transcription saved: /content/Power_English_Update (1).txt
Watching directory: /content/
🛑 Stopping real-time monitoring after 60 seconds.
